Arquivo de simulações utilizando sinais de áudio reais, retirados das basese MUSDB e VOICe Dataset. 

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display as ld
import musdb
from IPython.display import Audio
from scipy import stats
import soundfile as sf
import sox
import pandas as pd
import os
import mir_eval
from ast import literal_eval

In [3]:
def data_dict(tipo, mistura,algoritmo,nfontes,A, W,rate = None, white= None, amari = None, correlacao= None, corr_media = None,
         mses= None,mse_media = None, sdr=None, sir=None ,sar=None, sdr_media = None, sar_media = None, sir_media = None): 
    data_dict = {'Tipo':tipo, 'Mistura': mistura,'Algoritmo': algoritmo, 'Número de Fontes':nfontes,
                 'Matriz de mistura': A,'Matriz de separacao':W,
                 'Learning Rate':rate, 'White':white,
                 'Indice de Amari': amari,'Correlacao': correlacao,'Correlacao Media': corr_media,
                 'MSE': mses,'Media EQM': mse_media, 'SDR': sdr, 'SIR': sir, 'SAR': sar, 'SDR Media': sdr_media, 
                'SAR Media': sar_media, 'SIR Media': sir_media}
    #data_dict = {'Tipo':tipo, 'Mistura': mistura,'Algoritmo': algoritmo, 'Número de Fontes':nfontes,'Learning Rate':rate, 'White':white,
                 #'Índice de Amari': amari,'Correlacao': correlacao,'MSE': mses, 'SDR': sdr, 'SIR': sir, 'SAR': sar}
    return data_dict

In [4]:
#def source_dict(tipo, mistura, nfontes, s1kurt, s2kurt, s3kurt, s4kurt)

In [5]:
def save_data_csv(data_dict,create = False, path_to_csv = None):
    try:
        df = pd.read_csv(path_to_csv)
        df = df.append(data_dict, ignore_index = True)
        df.to_csv(path_to_csv, index = False)
    except(FileNotFoundError):
        if create:
            df = pd.DataFrame(columns = data_dict.keys())
            df = df.append(data_dict, ignore_index = True)
            df.to_csv(path_to_csv, index = False)
        else:
            raise('DataFrame não encontrado!')
    return df

In [6]:
def preprocess(tipo, arquivo):
    tfm = sox.Transformer()
    tfm.norm()
    tfm.rate(22050)
    pathold = 'E:/Audios/Antigos/'+tipo+arquivo
    pathnorm = 'E:/Audios/Novos/'+tipo+arquivo
    tfm.build_file(pathold+'/source1.wav', pathnorm+'/source1.wav')
    tfm.build_file(pathold+'/source2.wav', pathnorm+'/source2.wav')
    tfm.build_file(pathold+'/source3.wav', pathnorm+'/source3.wav')
    tfm.build_file(pathold+'/source4.wav', pathnorm+'/source4.wav')

In [7]:
def load(path, arquivo):
    """Carrega 4 arquivos de áudio localizados em pathmix."""
    
    pathmix = path+arquivo
    nome1 = pathmix+'/source1.wav'
    nome2 = pathmix+'/source2.wav'
    nome3 = pathmix+'/source3.wav'
    nome4 = pathmix+'/source4.wav'
    s1, sr = librosa.load(nome1, duration = 10)
    s2, sr = librosa.load(nome2, duration = 10)
    s3, sr = librosa.load(nome3, duration = 10)
    s4, sr = librosa.load(nome4, duration = 10)
    return s1,s2,s3,s4,sr

In [8]:
def whitening(X):
    """Realiza o branqueamento dos dados em X, retornando os dados branqueados em z junto 
    da matriz de branqueamento em Wwhite."""
    
    Xm = X - X.mean()
    covXm = np.cov(Xm)
    evalor, evetor = np.linalg.eig(covXm)
    wevalor = np.diag(1/(evalor**0.5))
    Wwhite = wevalor.dot(evetor.T) #matriz de branqueamento
    z = np.dot(Wwhite,Xm)
    return z, Wwhite

In [9]:
def normaliza(v):
    """Normaliza os dados recebidos."""

    vn = [k/np.std(k) for k in v]
    return np.array(vn)

In [10]:
def correlacao(Sn, Yn):
    """Realiza o cálculo da correlação de Pearson e separa os pares de maior correlação
    entre sinais originais e sinais estimados
    
    Parâmetros
    -----------------------------
    Sn: array
        Sinais originais normalizados
    Yn: array
        Sinais estimados normalizados
    
    Retorna
    -----------------------------
    allarrays: list 
        Lista contendo duas tuplas, a primeira com os pares de maior correlação, e a segunda com o índice
    de cada componente deste par. Exemplo: [(Sinal1, Sinal4), (0,3)]
    """
    
    allarrays = list()
    for p in range (0,len(Sn)): #Para comparar com S[0] e S[1]
        corrnova = 0
        corr = 0
        arrays = list()
        for o in range(0,len(Sn)): #Para comparar com Yn[0] e Yn[1]
            corrnova = stats.pearsonr(Sn[p],Yn[o])
            #print(f'Correlação de Pearson vale {stats.pearsonr(Sn[p], Yn[o])[0]} entre a {p+1} linha de S e a {o+1} linha de Y')
            if np.abs(corrnova[0]) > np.abs(corr): 
                corr = corrnova[0]
                arrays.clear()
                if corr < 0:
                    Yn[o] = Yn[o] * -1
                arrays.append(((Sn[p], Yn[o]),(p,o)))
        allarrays.append(arrays)
    return allarrays 
#Sn[p] e Y[o] são os pares de componentes com maior correlação
#retorna o vetor com os pares de maior correlaçao acompanhado de uma tupla (p,o) em que p é a posiçao de S[p] e o é a posição de Y[o]

In [11]:
def mse(pares, X):
    """Calcula o erro quadrático médio para pares de dados.
    
    Parâmetros
    -----------------------------
    pares: Lista contendo duas tuplas, sendo a primeira um par de variáveis e a segunda o índice dessas variáveis.
    
    Retorna
    -----------------------------
    mseall: Lista com o erro quadrático médio para cada par de dados.
    """
    
    mseall = list()
    msgrafs = list()
    for x in pares:
        mse = 0
        msgraf = list()
        for ponto in range (0, X.shape[1]):
            sqr = (x[0][0][0][ponto]- x[0][0][1][ponto])**2
            msgraf.append(sqr)
            mse += sqr
        msgrafs.append(msgraf)
        mse = mse/X.shape[1]
        posicao = x[0][1] #guarda posiçao das linhas de maior correlaçao
        mseall.append((mse, posicao))
    #for z in mseall:
       # print(f'O erro quadrático médio entre a {(z[1][0])+1}ª linha de S e a {(z[1][1])+1}ª linha de Y é de {z[0]}')
    return mseall, msgrafs
#retorna array com o erro quadratico medio e suas posições correspondentes 

In [12]:
def amari2(W, A):
    """Calcula o índice de amari para W e A """
    p = W.dot(A)
    C= D = 0
    for c in range(len(W)):
        a = b = 0
        for d in range(len(W)):
            a += (np.abs(p[c][d])/np.max(np.abs(p[c,:])))
            b += (np.abs(p[d][c])/np.max(np.abs(p[:,c])))
        C += a-1
        D += b-1
    E = C+D
    print(f'O índice de Amari obtido para W é de {E}')
    return E

In [13]:
def grafsinais(nome, mseall, Sn, Yn, mistura, tipo):
    """Plota os gráficos de erro quadrático médio.
    
    Parâmetros
    -----------------------------
    pares: Lista contendo duas tuplas, sendo a primeira um par de variáveis e a segunda o índice dessas variáveis.
    """
    
    for num, a in  enumerate(mseall):
        atual = str(num+1) #indice da fonte atual
        total = str(len(mseall)) #número total de fontes
        indexS = a[1][0]
        indexY = a[1][1]
        plt.figure(figsize = (14,5))
        plt.title(f'{tipo} - {mistura} - Comparação sinais S e Y - {nome} ({atual}/{total})')
        plt.plot(Sn[indexS])
        plt.plot(Yn[indexY])
        plt.legend(['Sinal Original', 'Sinal Separado'], loc = 'lower left')

In [14]:
def curtose(w, z, rate= 0, tipo = None):
    """Calcula a matriz de mistura com o algoritmo FastICA com curtose.
    
    Parâmetros
    -----------------------------
    w: Matriz de separação inicial.
    z: Sinal misturado branco.
    
    Retorna
    -----------------------------
    nome: Nome do algoritmo utilizado
    W: Matriz de separação
    """
    
    nome = "FastICA por Curtose"
    wlist = list()
    m = len(z)
    w1 = w
    want = 0
    cont = 0
    for p in range (1, m+1):
        while True:
            w1 = (((w1.T.dot(z)**3).dot(z.T)).T/z.shape[1])-3*w1
            w1 = w1/np.linalg.norm(w1)
            cont += 1
            if cont >= 2:
                want = w
                if np.allclose(w, want, atol = 0.000001):
                    break
            cont += 1  
        if p > 1:
            for j in range(1, p):
                a = w1.T.dot(wlist[j-1])
                w1 = w1 - (a[0]*wlist[j-1])
            w1 = w1/np.linalg.norm(w1)
        wlist.append(w1)
    W = list()
    for o in wlist:
        wfinal = list()
        for numP, p in enumerate(o):
            nw= p[0]
            wfinal.append(nw)
            if numP == len(o)-1:
                W.append(wfinal)            
    W = np.array(W)
    print(f'Número de iterações: {cont}')
    return nome, W, cont

In [15]:
#Algoritmo de FastICA com negentropia
def negentropia(w, z, rate = 0, tipo = None):
    """Calcula a matriz de separação com o algoritmo FastICA com negentropia.
    
    Parâmetros
    -----------------------------
    w: Matriz de separação inicial.
    z: Sinal misturado branco.
    
    Retorna
    -----------------------------
    nome: Nome do algoritmo utilizado
    W: Matriz de separação
    """
    
    nome = "FastICA por Negentropia"
    wlist = list()
    m = len(z)
    want = 0
    cont = 0
    for p in range (1, m+1):
        while True:
            w = (z.dot((np.tanh(w.T.dot(z)).T)))/z.shape[1] - (np.sum((1-(np.tanh(w.T.dot(z))**2))/z.shape[1])*w)
            w = w/np.linalg.norm(w)
            cont += 1
            if cont >= 2:
                want = w
                if np.allclose(w, want, atol = 0.000001):
                    break
        if p > 1:
            for j in range(1, p):
                a = w.T.dot(wlist[j-1])
                w = w- (a[0]*wlist[j-1])
            w = w/np.linalg.norm(w)
        wlist.append(w)
    W = list()
    for o in wlist:
        wfinal = list()
        for numP, p in enumerate(o):
            nw= p[0]
            wfinal.append(nw)
            if numP == len(o)-1:
                W.append(wfinal)            
    W = np.array(W)
    #print(f'Número de iterações: {cont}')
    return nome, W, cont

In [16]:
#Algoritmo Infomax
def infomax(W, X, rate, tipo = 'super'):
    """Calcula a matriz de separação com o algoritmo Infomax
    
    Parâmetros
    -----------------------------
    W: array
        Matriz de separação inicial
    X: array
        Mistura de sinais
    rate: float
        Taxa de aprendizagem
 
    tipo : str
        Define qual variação do algoritmo será utilizada (sup/super/extended)
        
    Retorna
    -----------------------------
    nome: str 
        Nome do algoritmo utilizado
    W: array
        Matriz de separação
    """
    
    lenght_X = len(X)
    #Wlist = list()
    nome = "Infomax " + tipo
    I = np.eye(lenght_X, lenght_X)
    cont = None
    for k in range (0,X.shape[1]):
        y = W.dot(X[:,k])
        if tipo == 'super':
            fy = np.array([np.tanh(y[m]) for m in range (lenght_X)])
        elif tipo == 'sub':
            fy = np.array([y[m] - np.tanh(y[m]) for m in range (lenght_X)])
        W = W + rate*np.dot((I-np.outer(fy,y)),(W))
      #  Wlist.append(W)
    #for i in range(0, W.shape[0]):
        #w_pesos = list()
        #for p in Wlist:
         #   w_pesos.append(p[i])
        #plt.figure()
       # plt.plot(w_pesos)
    return nome, W, cont

In [17]:
def correqm_mean(correlacoes_eqms): #media de correlacoes OU eqm
    sum_corr = 0
    for corr_it in range(0, len(correlacoes_eqms)):
        sum_corr+=abs(correlacoes_eqms[corr_it])
    mean_c = sum_corr/len(correlacoes_eqms)
    return mean_c

In [19]:
def teste(tipo, mistura ,S, X, algoritmo, w, A,save = False, rate = 0.001, white = False , tipoinfomax = 'super', path_to_csv = None):
    """Aplica o ICA em uma determinada mistura de acordo com o algoritmo desejado
    
    Parâmetros
    -----------------------------
    S: array 
        Sinais originais
    X: array
        Sinais misturados
    algoritmo: função
        nome da função do algoritmo desejado
    w: array
        matriz de separação inicial
    A: array 
        Matriz de mistura
    rate: float 
        taxa de aprendizagem
    white: Bool
        False caso não se deseje fazer o branqueamento da mistura
        
    Retorna
    -----------------------------
    Yn: array
        Sinais estimados
    
    """
    try:
        Sn = normaliza(S)
        nfontes = len(Sn)
        if white:
            X, wwhite = whitening(X)
        nome, W, steps = algoritmo(w, X, rate, tipo = tipoinfomax)
        try:
            y = W.dot(X)
        except(ValueError):
            y = np.transpose(W).dot(X)
        print(f'Nome do algoritmo: {nome}')
        Yn = normaliza(y)
        if white:
            W = W.dot(wwhite)
        amari = amari2(W,A)
        pares = correlacao(Sn,Yn)
        correlacoes = [stats.pearsonr(a[0][0][0], a[0][0][1])[0] for a in pares]
        corr_media = correqm_mean(correlacoes)
        mses, graf_mse = mse(pares, X)
       # grafmse(nome, graf_mse, Sn, Yn, mistura)
       # grafsinais(nome, mses, Sn, Yn, mistura)
        eqm_vect = [eqm[0] for eqm in mses]
        eqm_media = correqm_mean(eqm_vect) 
        sdr, sir, sar, perm= mir_eval.separation.bss_eval_sources(Sn, Yn, compute_permutation = True)
        sdr_media =correqm_mean(sdr)
        sar_media =correqm_mean(sar)
        sir_media =correqm_mean(sir)
        #print(f'SDR: {sdr}\nSIR: {sir}\nSAR: {sar}')
        A_dict = A.reshape(1,A.shape[0]*A.shape[1])
        W_dict = W.reshape(1, A.shape[0]*W.shape[1])
        A_dict = A_dict[0]
        W_dict = W_dict[0]
        if save == True:
            ddict = data_dict(tipo, mistura,nome,nfontes, A_dict, W_dict,rate, white, amari, correlacoes,corr_media,eqm_vect,
                              eqm_media,sdr,sir, sar, sdr_media, sar_media, sir_media)
            save_data_csv(ddict, create = True, path_to_csv = path_to_csv)
        print()
        return Yn, pares
    except:
        if save == True:
            ddict = data_dict(tipo, mistura,nome,nfontes, A_dict, W_dict,rate)
            save_data_csv(ddict, create = True, path_to_csv = path_to_csv)
        print('Algoritmo não rodou com sucesso')
        pass

In [20]:
def simula(tipo, savewa = False,do_white_fastica = True, do_white_infomax = False, pre_process = True, save = False, path_to_csv = None):
    """Função para realizar as simulações, variando a mistura de fontes e o número de fontes a ser analisado.
        
    Parâmetros
    -----------------------------
    tipo : str
        tipo de fontes a ser utilizado
    pre_process : bool
        define se o pré processamento deverá ser realizado
    """
    for numero in range(1,6):
        tipo1 = tipo+'/'
        num = str(numero)
        arquivo = "Mistura"+num
        path = 'E:/Audios/Novos/'+tipo1
        
        try:
            s1, s2, s3, s4, sr = load(path, arquivo)
        except(FileNotFoundError):
            if pre_process:
                preprocess(tipo1,arquivo)
                s1, s2, s3, s4, sr = load(path, arquivo)
            else:
                raise('Path não existe!')

        Slist = [s1, s2]
        print(arquivo)

        for n_sources in range(2,3):
            print(f'Número de fontes: {n_sources}')
            S = np.array(Slist)
            lenght_S = len(S)
            A = np.random.randn(lenght_S, lenght_S)
            X = A.dot(S)
            for rept in range(0,20):
                w = np.random.randn(len(X), 1)
                w = w/np.linalg.norm(w)
                winfomax = np.random.randn(lenght_S, lenght_S)
                teste(tipo, arquivo, S, X, curtose, w,A, save= save, white = do_white_fastica, path_to_csv = path_to_csv)
                teste(tipo, arquivo, S, X, negentropia, w, A,save = save, white = do_white_fastica, path_to_csv = path_to_csv)
                teste(tipo, arquivo, S, X, infomax, winfomax, A, save = save, rate = 0.001, white=do_white_infomax, tipoinfomax = 'super', path_to_csv = path_to_csv)
                teste(tipo, arquivo, S, X, infomax, winfomax, A,  save = save, rate = 0.001, white=do_white_infomax, tipoinfomax = 'sub', path_to_csv = path_to_csv)
            if n_sources == 2:
                Slist.append(s3)
            if n_sources == 3:
                Slist.append(s4)

In [21]:
def orgcsv(path):
    a = pd.read_csv(path)
    a = a.sort_values(by=['Algoritmo', 'Mistura'])
    a_edit = a.drop('Matriz de mistura', 1)
    a_edit = a_edit.drop('Matriz de separacao', 1)
    a_edit = a_edit.drop('Correlacao', 1)
    a_edit = a_edit.drop('MSE', 1)
    a_edit = a_edit.drop('White',1 )
    a_edit = a_edit.drop('Learning Rate',1 )
    return a_edit

In [29]:
def mist_mean_csv(path, df):
    algoritmos = df['Algoritmo'].unique()
    misturas = df['Mistura'].unique()
    for alg in algoritmos:
        for mst in misturas:
            sum_cor = sum_eqm = sum_amr = sum_sdr = sum_sir = sum_sar = n_misturas =0
            for nm,(index, row) in enumerate(df.iterrows()):
                if row['Algoritmo'] == alg and row['Mistura'] == mst:
                    n_misturas += 1
                    sum_cor += abs(row['Correlacao Media'])
                    sum_eqm += row['Media EQM']
                    sum_amr += row['Indice de Amari']
                    sum_sdr += row['SDR Media']
                    sum_sar += row['SAR Media']
                    sum_sir += row['SIR Media']
                if ((row['Algoritmo'] != alg or row['Mistura'] != mst) and sum_cor != 0 and sum_eqm != 0 and sum_amr != 0) or (row['Algoritmo'] == alg and (nm+1) == len(df.index)):
                    corr_mistura = sum_cor/n_misturas
                    eqm_mistura = sum_eqm/n_misturas
                    amr_mistura = sum_amr/n_misturas
                    sdr_mistura = sum_sdr / n_misturas
                    sar_mistura = sum_sar / n_misturas
                    sir_mistura = sum_sir/n_misturas
                    sum_cor = sum_eqm = sum_amr = sum_sdr = sum_sir = sum_sar  = 0
                    dic_mean = {'Tipo':row['Tipo'],'Algoritmo': alg, 'Mistura':mst, 
                                'Número de Fontes': row['Número de Fontes'], 'Indice de Amari media': amr_mistura,
                               'Correlacao Media': corr_mistura, 'EQM Media': eqm_mistura, 'SDR Media': sdr_mistura,
                               'SAR Media': sar_mistura, 'SIR Media': sir_mistura}
                    save_data_csv(dic_mean, create=True, path_to_csv = path)
                    break
    csv = pd.read_csv(path)
    return csv

In [23]:
def alg_mean_csv(path, df): #calcula a média de valor para cada algoritmo
    algoritmos = df['Algoritmo'].unique()
    data_dict = dict()
    for n_it, alg in enumerate(algoritmos):
        amr_list= list()
        corr_list=list() 
        eqm_list = list()
        sum_cor = sum_eqm = sum_amr = n_misturas = sum_sdr = sum_sir = sum_sar = 0
        n = df['Algoritmo'].value_counts().to_dict()
        n_alg = n[alg]
        nnn = n_it+1
        for nm,(index, row) in enumerate(df.iterrows()):
            if row['Algoritmo'] == alg:
                n_misturas += 1
                sum_cor += abs(row['Correlacao Media'])
                sum_eqm += row['EQM Media']
                sum_amr += row['Indice de Amari media']
                sum_sdr += row['SDR Media']
                sum_sar += row['SAR Media']
                sum_sir += row['SIR Media']
                amr_list.append(abs(row['Correlacao Media']))
                corr_list.append(row['EQM Media'])
                eqm_list.append(row['Indice de Amari media'])
            if (row['Algoritmo'] != alg and sum_cor != 0 and sum_eqm != 0 and sum_amr != 0) or (row['Algoritmo'] == alg and n_alg == n_misturas and nnn == len(algoritmos)):
                corr_mistura = sum_cor/n_misturas
                eqm_mistura = sum_eqm/n_misturas
                amr_mistura = sum_amr/n_misturas
                sdr_mistura = sum_sdr/n_misturas
                sar_mistura = sum_sar/n_misturas
                sir_mistura = sum_sir/n_misturas
                eqm_array = np.array(eqm_list)
                amr_array = np.array(amr_list)
                corr_array = np.array(corr_list)
                alldata_alg = (eqm_array, amr_array, corr_array)
                dic_mean = {'Tipo':row['Tipo'],'Algoritmo': alg, 
                            'Número de Fontes': row['Número de Fontes'], 'Indice de Amari media': amr_mistura,
                           'Correlacao Media': corr_mistura, 'EQM Media': eqm_mistura, 'Indice de Amari std': np.std(amr_list)
                           , 'EQM std': np.std(eqm_list), 'Correlacao std': np.std(corr_list), 'SDR Media': sdr_mistura,
                           'SAR Media': sar_mistura, 'SIR Media': sir_mistura}
                save_data_csv(dic_mean, create=True, path_to_csv = path)
                data_dict[alg] = alldata_alg
                sum_cor = sum_eqm = sum_amr = 0
                break 
    csv = pd.read_csv(path)
    csv['Indice de Amari std'] = np.std(amr_array)
    csv['EQM std'] = np.std(eqm_array)
    csv['Correlacao std'] = np.std(corr_array) #dict ={algoritmo: [eqm, amari, correlacao]}
    return csv, data_dict

In [24]:
def org_data(path_to_csv, path1 = None, path2 = None):
    df = orgcsv(path_to_csv)
    df_new = mist_mean_csv(path1, df)
    df_new2, data_dict = alg_mean_csv(path2, df_new)
    return df_new2, df_new, data_dict

In [25]:
simula(tipo = 'vehicle', save = True, path_to_csv = 'audio_vehicle.csv')

Mistura1
Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.031686644063851244

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.02819946025302067

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.392900337502182

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.4928663987127635

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.03596067824589144

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.023950790267586175

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.341701309560417

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.5301612677942633

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.01831037362415744

Nome do algoritmo: FastICA por Negentropia
O índice d


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.331782852920098

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.4154149522353119

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.7765373436714988

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.1147825702237735

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.3317767473349353

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.9094768939756863

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.00848016229308346

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.10172402790230906

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.331774881488342

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.4487869735495431

Número de iterações: 4
Nome


Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.1253964706560393

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.1000054507917576

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.7423312230960022

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.4961030539397746

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 2.222688706066088

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.534388608051307

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.7425130440537981

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.6033196619161534

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.5128482340695455

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.456922


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.819119071265275

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.249474275153289

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.03629125175873327

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.022712696188502024

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.8208188277136284

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.5568774405422745

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.1587610785341238

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.1774161114433912

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.8224002207488812

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.3593866913642674

Número de iterações: 4
Nom


Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.005974906993528428

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.7568590044656867

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.2702051398085168

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.18261850398324841

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.7306139819022179

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.7100003265077852

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.3331033967008195

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.014708355253422623

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.005222060074462576

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.7154869911842472

Nome do 

In [26]:
simula(tipo = 'indoor', save = True, path_to_csv = 'audio_indoor.csv')

Mistura1
Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.042441953097125396

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.030200962305618972

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.0103859494435596

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.0065617564059322

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.027393213323990917

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.04868004773092527

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.0059605847449093

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.520611525498584

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.12463590221503895

Nome do algoritmo: FastICA por Negentropia
O índice


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.2569874167072626

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.5815787429306571

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.06936810207261734

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.032465508120729014

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.24213721847865277

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.4939071364705276

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.0720588818733916

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.08513262695708224

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.599451774975811

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.4433929050607242

Número de iterações: 4



Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.0849264303763102

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.886401111429823

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.10168954518950835

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.08987340348241601

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.11149322847619003

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.886389415114178

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.10183359578762818

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.41779847316788365

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.6290116360572233

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.8863562325999323

Nome do algo


Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.283895807657986

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.5130420455107745

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.9382079693573189

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.420194342443051

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2352603324280396

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.026223918364387977

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.013600290305930463

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.42013345277622305

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2589276349582246

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.0078846073013


Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.13215655612106003

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.245927334440256

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.458099744610398

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.1621616164617277

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.0756156752449346

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.7073634095490116

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.1381526254456522

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.06289716842726878

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.393276622380452

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.476108007678681

Nome do algoritmo

In [27]:
simula(tipo = 'outdoor', save = True, path_to_csv = 'audio_outdoor.csv')

Mistura1
Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.04291612734755201

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.06288392748428473

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.9081269799143064

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.38541484726974806

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.7754671890258114

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.0264095180344412

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.8450475438006855

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.49667723867886115

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.8818486642484549

Nome do algoritmo: FastICA por Negentropia
O índice de


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.4455161501738507

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.3607526508275365

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.06300664111193965

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.0027374771972989986

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.4332061310934074

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.3670588173779772

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.11448122647742642

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.12053030993802061

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 3.439278657053821

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.35602225832662504

Número de iterações: 


Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.29124542167195

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.49974647121941884

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2732133705914315

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.087848998036538

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.19231513489624796

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.42125956396111364

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.355137209100089

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.10100016601073936

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.09791345406997887

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.5637384334261626

Nome do algori


Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.2729554337943356

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.34858868748417327

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.3883009871273484

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.47534363847667827

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.2730992568553132

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.3522458571438811

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.3883464476684093

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.4615377605559645

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.05596424527599608

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.4


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.8132911785938717

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2532774554317134

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.02995371795830093

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.07025026799154488

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.8126840791130947

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2521360392408294

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.30826676969438926

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.3987513948094774

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 1.8129409587032408

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.2536306095234746

Número de iterações: 4
N

In [28]:
simula(tipo = 'musicas', save = True, path_to_csv = 'audio_musicas.csv')

Mistura1
Número de fontes: 2
Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.2089916435426824

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.11922554842573785

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.15010330915908132

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.6511476131343832

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.2355410688806967

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.13031940637434358

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.14628826471380307

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.6478647289951562

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.2633216834058043

Nome do algoritmo: FastICA por Negentropia
O índice de


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.3554238698492953

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 3.126262930586486

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.033073068575897535

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.014806623909128902

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.3553536587910622

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 3.123211831348928

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.03237984726872667

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.03301504271212963

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.3554328142577128

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 3.1359518068843784

Número de iterações: 4



Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.985421994204489

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 2.308158790286886

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.37458184878786693

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.824185358005442

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.15473628381337212

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.2677119638542833

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.3745818487878887

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 2.824097230602003

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.0407067538555832

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.033344


Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.503608524945344

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.5354525413357516

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 1.1072488087068275

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 1.3401199420276797

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.5036085249490148

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.53563737160989

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.05138148622805794

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.16506219351750206

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 2.5036085250057987

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 1.5357740733151228

Número de iterações: 4
Nome 


Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.03226537833494647

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.4040606724526341

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.5308865938201022

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.0584719762657524

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.07021582699574425

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.4040583554562611

Nome do algoritmo: Infomax sub
O índice de Amari obtido para W é de 0.5387398710976503

Número de iterações: 4
Nome do algoritmo: FastICA por Curtose
O índice de Amari obtido para W é de 0.0957547918341648

Nome do algoritmo: FastICA por Negentropia
O índice de Amari obtido para W é de 0.03418937933307187

Nome do algoritmo: Infomax super
O índice de Amari obtido para W é de 0.40407416892385384

Nome do alg

In [ ]:
#df_test, df_test1, data_dic = org_data('', 'd', '')

In [32]:
df_indoor, df_indoor1, dic_indoor = org_data('audio_indoor.csv', 'test_ind_2', 'teste_ind_3')

C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a.drop('Matriz de mistura', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Matriz de separacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Correlacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('MSE', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_

In [33]:
pd.read_csv('teste_ind_3')

,Tipo,Algoritmo,Número de Fontes,Indice de Amari media,Correlacao Media,EQM Media,Indice de Amari std,EQM std,Correlacao std,SDR Media,SAR Media,SIR Media
0,indoor,FastICA por Curtose,2,0.295384,0.988169,0.683544,0.007197,0.069810,0.445381,28.343896,38.218817,33.672007
1,indoor,FastICA por Negentropia,2,0.384595,0.984309,0.463638,0.007201,0.131318,0.343656,26.556560,38.506089,31.785072
2,indoor,Infomax sub,2,1.279983,0.908100,0.502310,0.078265,0.710308,0.671631,14.272714,152.016492,14.272714
3,indoor,Infomax super,2,1.740047,0.829880,0.716746,0.142111,0.882097,0.420832,11.827245,151.985578,11.827245


In [30]:
df_musicas, df_musicas1, dic_musicas = org_data('audio_musicas.csv', 'test_msc_2', 'teste_msc_3')

C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a.drop('Matriz de mistura', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Matriz de separacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Correlacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('MSE', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_

In [31]:
pd.read_csv('teste_msc_3')

,Tipo,Algoritmo,Número de Fontes,Indice de Amari media,Correlacao Media,EQM Media,Indice de Amari std,EQM std,Correlacao std,SDR Media,SAR Media,SIR Media
0,musicas,FastICA por Curtose,2,0.349786,0.987106,0.353631,0.007113,0.173566,0.214463,32.088511,65.089104,32.718186
1,musicas,FastICA por Negentropia,2,0.492322,0.978120,0.332439,0.010768,0.183880,0.136955,30.218550,65.171485,30.635431
2,musicas,Infomax sub,2,1.933060,0.851567,0.835733,0.123373,0.933864,0.544356,11.794662,151.976000,11.794662
3,musicas,Infomax super,2,0.764900,0.951194,0.645681,0.076774,0.872644,0.562491,21.274685,151.965708,21.274685


In [34]:
df_outdoor, df_outdoor1, dic_outdoor = org_data('audio_outdoor.csv', 'test_out_2', 'teste_out_3')

C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a.drop('Matriz de mistura', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Matriz de separacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('Correlacao', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_18220/3469787827.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  a_edit = a_edit.drop('MSE', 1)
C:\Users\aleke\AppData\Local\Temp/ipykernel_

In [35]:
pd.read_csv('teste_out_3')

,Tipo,Algoritmo,Número de Fontes,Indice de Amari media,Correlacao Media,EQM Media,Indice de Amari std,EQM std,Correlacao std,SDR Media,SAR Media,SIR Media
0,outdoor,FastICA por Curtose,2,0.502681,0.978840,0.572199,0.023317,0.408654,0.429160,26.164124,50.004071,27.051077
1,outdoor,FastICA por Negentropia,2,0.606444,0.974912,0.506866,0.021537,0.395108,0.258870,25.117972,49.931227,26.254508
2,outdoor,Infomax sub,2,1.162009,0.911054,1.058983,0.106924,0.909921,0.563321,17.023119,151.976937,17.023119
3,outdoor,Infomax super,2,2.133275,0.811846,0.465328,0.121836,0.967253,0.319683,9.755918,151.989728,9.755918
